<a href="https://colab.research.google.com/github/shpotes/clustering/blob/master/notebooks/experiments/substractive-clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!git clone https://github.com/shpotes/clustering
#!pip install datasets optuna
%cd clustering

/content/clustering


In [2]:
import pathlib
from datasets import load_dataset
import pandas as pd
import numpy as np
import optuna
from src import Substractive, KMeansPlusPlus
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score

PyTorch version 1.6.0+cu101 available.
TensorFlow version 2.3.0 available.


In [3]:
dataset = load_dataset('rotten_tomatoes')

Checking /root/.cache/huggingface/datasets/b3267cede0118f731406527d730b20a47827f64e15eaece772b81c2ec6c4e129.75128cbb0bbc33ddc2b7299d2a29d743223662ec632601fc16d2dea19bcb1072.py for additional imports.
Found main folder for dataset https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/rotten_tomatoes/rotten_tomatoes.py at /root/.cache/huggingface/modules/datasets_modules/datasets/rotten_tomatoes
Found specific version folder for dataset https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/rotten_tomatoes/rotten_tomatoes.py at /root/.cache/huggingface/modules/datasets_modules/datasets/rotten_tomatoes/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a
Found script file from https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/rotten_tomatoes/rotten_tomatoes.py to /root/.cache/huggingface/modules/datasets_modules/datasets/rotten_tomatoes/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a/rotten_tomatoes.py
Found d

In [4]:
data_dir = pathlib.Path('repr')
embedding = np.load(data_dir / 'word2vec' / 'en_core_web_sm_validation.npy')

In [5]:
embedding.shape

(1066, 96)

In [6]:
r_a = 2 #trail.suggest_uniform('r_a', 1e-2, 1e2)

sub = Substractive(r_a=r_a, tol=0.1)
labels = sub.fit_predict(embedding)

ConcretizationTypeError: ignored

In [ ]:
def objective(trail):
    r_a = trail.suggest_loguniform('r_a', 1e-1, 1e1)
    sub = Substractive(r_a=r_a, tol=0.1)
    labels = sub.fit_predict(embedding)
    
    if len(np.unique(labels)) == 1:
        return -2

    return davies_bouldin_score(embedding, labels)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)